# Exoplanet Data Analysis
## Python Workbook



#### Firstly, import the necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import matplotlib.colors as mccolours
from matplotlib.colors import LogNorm
import matplotlib.ticker as ticker
from matplotlib.transforms import Bbox
import matplotlib.cm as cm
import matplotlib.collections as collections
import matplotlib.lines as lines
from pandas import read_csv
from pandas import Series
from astropy import coordinates
from astropy import units
from astropy.table import QTable, vstack, Column, hstack
from astroquery.simbad import Simbad
from astroquery import gaia as Gaia
from astroquery.gaia import GaiaClass

## Graph Plotting Utilities

In [ ]:
def readData(filename, headerRow=290):
    '''
    Read csv data.
    '''
    return read_csv(filename, header=headerRow)

def plotScatterGraphOfData(szTitle, xAxis, yAxis, xData, yData, xError, yError, logariseXAxis, logariseYAxis, colours = np.array([]), colourAxisLabel = "", bColourLoggersised=False, sizes = np.array([]), tickSeperation = 0.5):
    bWasColourData = False

    plt.rcParams['font.family'] = "Times New Roman"
    plt.rcParams['font.size'] = 10
    plt.rcParams['axes.labelsize'] = 10 # 10  # Size of axis labels
    plt.rcParams['axes.titlesize'] = 10  # Size of title
    plt.rcParams['xtick.labelsize'] = 10 # 8  # Size of x-axis tick labels
    plt.rcParams['ytick.labelsize'] = 10 # 8  # Size of y-axis tick labels
    plt.rcParams['legend.fontsize'] = 10 # 8  # Size of legend font
    plt.rcParams['figure.figsize'] = [3.5, 2.5] # [3.487, 2.155]  # Figure dimension in inches (1-column width)
    plt.rcParams['figure.dpi'] = 300  # Figure resolution
    plt.rcParams['savefig.dpi'] = 300  # Resolution of saved figure
    plt.rcParams['text.usetex'] = True  # Use LaTeX for text rendering

    # Sets the colours of the data points if no colour data was specified
    if colours.size == 0:
        colours = np.full(shape=xData.size , fill_value="b")
    else:
        bWasColourData = True
    
    # Creates the colour map
    if (bColourLoggersised and bWasColourData):
        myNorm = LogNorm(vmin=colours.min(), vmax=colours.max())
        colourMap = cm.ScalarMappable(norm=myNorm, cmap="Spectral")
    else:
        myNorm = None
        colourMap = cm.ScalarMappable(cmap="Spectral")
    
    colourMap.set_array(colours)

    # Extracts the array of colours from the colour map
    rgbaArray = cm.ScalarMappable.to_rgba(colourMap, colourMap.get_array())

    # Sets the sizes of the data points if no size data was specified
    if sizes.size == 0:
        sizes = np.full(shape=xData.size , fill_value=6)

    # Create the figure and get the main axes
    figure = plt.figure(dpi=400)
    mainAxes = figure.gca()

    # Sets the aspect ratio of the atual graph (not including the error bar)
    mainAxes.set_box_aspect(1.0)

    # Create the scatter graph onto the main axes
    mainAxes.scatter(xData, yData, s = sizes, c = rgbaArray, cmap = colourMap, marker="o")

    # Adds error bars
    if bWasColourData == True:
        # Creates error bars and extracts their details
        dataLines, capLines, barLines = mainAxes.errorbar(xData, yData, xerr=xError, yerr=yError, fmt="none", elinewidth=0.6, capsize=0, barsabove=True, capthick=0.5)

        # Set's the colour of the error bars
        barLines[0].set_colors(rgbaArray) # Does the x lines
        barLines[1].set_colors(rgbaArray) # Does the y lines
        
        # Unfortunetly the below code does not do anything. You may try. Due to this we cannot have coloured caps, so we choose to not include them
        #  capLines[0].set_color((rgbaArray, rgbaArray))
        #  capLines[1].set_color((rgbaArray, rgbaArray))

    else:
        mainAxes.errorbar(xData, yData, xerr=xError, yerr=yError, fmt="none", ecolor='black', elinewidth=0.6, capsize=0, barsabove=True, capthick=0.5)

    # Adds a title
    mainAxes.set_title(szTitle)

    # Customise the axes
    if logariseXAxis:
        mainAxes.set_xscale("log")
    if logariseYAxis:
        mainAxes.set_yscale("log")
    mainAxes.set_xlabel(xAxis)
    mainAxes.set_ylabel(yAxis)

    mainAxes.xaxis.set_major_locator(ticker.MultipleLocator(tickSeperation))
    mainAxes.xaxis.set_minor_locator(ticker.MultipleLocator(tickSeperation/5.0))

    mainAxes.tick_params(axis='both', which = 'both', direction='in', top = True, right = True)
    mainAxes.tick_params(axis='both', which = 'major', length = 2, width = 0.5)
    mainAxes.tick_params(axis='both', which = 'minor', length = 1, width = 0.5)

    # Creates the colour bar
    if bWasColourData:
        colourBarAxes = figure.add_axes((0.85, 0.12, 0.05, 0.75))
        figure.colorbar(colourMap, cax=colourBarAxes)
        colourBarAxes.set_ylabel(colourAxisLabel)
        if (bColourLoggersised):
            colourBarAxes.set_yscale("log")

        colourBarAxes.tick_params(axis='both', which = 'major', length = 2, width = 0.5)
        colourBarAxes.tick_params(axis='both', which = 'minor', length = 1, width = 0.5)

    # Returns the entire figure
    return figure

# Saves the graph in the specified format. We recommend "pdf". This ensures that no quality is lossed,
# compatibility is maximised, text can be highlighted and copied etc., and file size is minimal
def saveGraph(saveFileName, szFormat="pdf"):
        plt.savefig(saveFileName, format=szFormat)
        print("Graph saved as a", szFormat, "file with name:", saveFileName)


# Create a colour dictionary of detection methods from the exoplanet archive data
def detectionMethodColourDictionary(planetData):
    detectionMethods = planetData["discoverymethod"].to_numpy()
    detectionColours = detectionMethods.copy()
    i = 0
    for detectionMethod in detectionMethods:
        if detectionMethod == "Radial Velocity":
            detectionColours[i] = "r"
        elif detectionMethod == "Transit":
            detectionColours[i] = "b"
        elif detectionMethod == "Imaging":
            detectionColours[i] = "g"
        else:
            detectionColours[i] = "m"
        i = i + 1
    
    return detectionColours



## Data Analysis Utilities

In [ ]:
def getGAIAData(coordsAndMagnitudes):
    "Retrives data on a single GAIA object"

    # Put the coordinates into a skycoord object
    mySkyCoords = coordinates.SkyCoord(ra=coordsAndMagnitudes["Right Ascension"], dec=coordsAndMagnitudes["Declination"])

    # Search for the system in the GAIA archive
    print("Querying the GAIA archive")
    myStarProperties = Gaia.Gaia.query_object_async(mySkyCoords, width=8*units.arcsecond, height=8*units.arcsecond)

    # Further filter using magnitudes
    myStarProperties = myStarProperties[myStarProperties['phot_g_mean_mag'] > (coordsAndMagnitudes["GAIA Magnitude"] - 0.5)]
    myStarProperties = myStarProperties[myStarProperties['phot_g_mean_mag'] < (coordsAndMagnitudes["GAIA Magnitude"] + 0.5)]

    # parallaxErrorMyStar = myStarProperties['parallax_error']
    return myStarProperties

def getGAIADataMultipleStars2(coordsAndMagnitudes, width = 8, height = 8):
    "Returns an AstroPy QTable for a list of all objects matching those in the list of objects passed to this function"
    return(getGAIADataMultipleStars(coordsAndMagnitudes['Right Ascension'], coordsAndMagnitudes['Declination'], coordsAndMagnitudes['GAIA Magnitude'], width, height))


def getGAIADataMultipleStars(RAs, Decs, gaiaMagnitudes, width = 8, height = 8):
    "Returns an AstroPy QTable for a list of all objects matching those in the list of objects passed to this function"
    # Defines the width of the box to search
    width=width*units.arcsecond
    height=height*units.arcsecond

    # If the code breaks at this line, please post an issue on the github page (https://github.com/george112n/How-Metallicity-Affects-Exoplanets/issues/new)
    
    # Define the table names of the GAIA archive - Up to date on 13/03/24. As seen DR3 is the latest release on this date.
    gaiaSourceTable = "gaiadr3.gaia_source"
    gaiaAstrophysicalParametersTable = "gaiadr3.astrophysical_parameters"

    # Defines the list of coloumns that you are interested in from the gaiaSourceTable
    columns = "{gaiaSourceTable}.ra, {gaiaSourceTable}.ra_error, {gaiaSourceTable}.dec, {gaiaSourceTable}.dec_error, {gaiaSourceTable}.pmra, {gaiaSourceTable}.pmra_error, {gaiaSourceTable}.pmdec, {gaiaSourceTable}.pmdec_error, {gaiaSourceTable}.radial_velocity, {gaiaSourceTable}.radial_velocity_error, {gaiaSourceTable}.parallax, {gaiaSourceTable}.parallax_error, {gaiaSourceTable}.phot_g_mean_mag"
    
    # Defines the list of coloumns that you are interested in from the AstroPhysicalParams
    columns = columns + ", {AstroPhysicalParams}.mh_gspphot, {AstroPhysicalParams}.mh_gspphot_upper, {AstroPhysicalParams}.mh_gspphot_lower, {AstroPhysicalParams}.alphafe_gspspec, {AstroPhysicalParams}.alphafe_gspspec_lower, {AstroPhysicalParams}.alphafe_gspspec_upper"
    
    # Substitutes the placeholder for the table names with the real table names
    columns = columns.format(**{'gaiaSourceTable': gaiaSourceTable, 'AstroPhysicalParams': gaiaAstrophysicalParametersTable})
    
    ### The following lines of code create a single query for all of the objects you are interested in ###
    query = """
        SELECT
        {row_limit}""" #   DISTANCE(
        #     POINT('ICRS', {ra_column}, {dec_column}),
        #     POINT('ICRS', {ra}, {dec})
        # ) as dist,"""
    query = query + """
        {columns}
        FROM
        {gaiaSourceTable}
        JOIN {AstroPhysicalParams} ON {AstroPhysicalParams}.source_id = {gaiaSourceTable}.source_id
        WHERE """
    for i in range(len(RAs) - 1):
        query = query + """(1 = CONTAINS(
            POINT('ICRS', {gaiaSourceTable}.{ra_column}, {gaiaSourceTable}.{dec_column}),
            BOX('ICRS',
                """
        query = query + str(RAs[i].value) + ", "
        query = query + str(Decs[i].value) + """,
            {width},
            {height}
                )
            )
            AND ({gaiaSourceTable}.phot_g_mean_mag BETWEEN """ +str(gaiaMagnitudes[i]-0.5) +" AND " +str(gaiaMagnitudes[i]+0.5) +""")
        )
        OR
        """
    
    query = query + """(1 = CONTAINS(
        POINT('ICRS', {gaiaSourceTable}.{ra_column}, {gaiaSourceTable}.{dec_column}),
        BOX('ICRS',
            """
    query = query + str(RAs[len(RAs) - 1].value) + ", "
    query = query + str(Decs[len(RAs) - 1].value) + """,
        {width},
        {height}
        )
    )
    AND({gaiaSourceTable}.phot_g_mean_mag BETWEEN """ +str(gaiaMagnitudes[len(RAs) - 1]-0.5) +" AND " +str(gaiaMagnitudes[len(RAs) - 1]+0.5) +""")
    )"""

    query = query.format(**{'row_limit': "",
                      'ra_column': GaiaClass.MAIN_GAIA_TABLE_RA, 'dec_column': GaiaClass.MAIN_GAIA_TABLE_DEC,
                      'columns': "*" if columns == "" else columns,
                      'table_name': gaiaSourceTable+","+gaiaAstrophysicalParametersTable,
                      'gaiaSourceTable': gaiaSourceTable, 'AstroPhysicalParams': gaiaAstrophysicalParametersTable,
                      'width': width.to(units.deg).value , 'height': height.to(units.deg).value})
    
    # print(query)

    # Queries the GAIA archive
    print("Querying the GAIA archive")
    job = Gaia.Gaia.launch_job_async(query, verbose=True)
    
    # Use job. to retrieve more information about the GAIA job

    # Returns the job results
    return(job.get_results())

def getAllGAIAObjects():
    """Queries the top 10000 GAIA objects from the gaia_source table\n
    Use this for testing your connection to the GAIA archive"""
    
    print("Querying the GAIA archive")
    job = Gaia.Gaia.launch_job_async("select top 10000 ra, ra_error, dec, dec_error, pmra, pmra_error, pmdec, pmdec_error, radial_velocity, radial_velocity_error, parallax, parallax_error, mh_gspphot, mh_gspphot_upper, mh_gspphot_lower from gaiadr3.gaia_source")
    return(job.get_results())

def computeGalacticVelocity(GAIAStar):
    "Calculates the galatic velocity of a star given a row of an AstroPy table retrieved from the GAIA Exoplanet archive"
    # Extract some star data from the GAIA archive results
    rightAscension = GAIAStar['ra']
    rightAscensionError = GAIAStar['ra_error']

    declenation = GAIAStar['dec']
    declenationError = GAIAStar['dec_error']

    properMotionRA = GAIAStar['pmra']
    properMotionRAError = GAIAStar['pmra_error']

    properMotionDec = GAIAStar['pmdec']
    properMotionDecError = GAIAStar['pmdec_error']

    radialVelocity = GAIAStar['radial_velocity']
    radialVelocityErr = GAIAStar['radial_velocity_error']

    # And the paralax
    parallax = GAIAStar['parallax']
    parallaxError = GAIAStar['parallax_error']

    return(GalaVel(rightAscension, declenation, properMotionRA, properMotionRAError, properMotionDec, properMotionDecError, radialVelocity, radialVelocityErr, parallax, parallaxError))

# This function was entirely provided by Thomas Wilson
def GalaVel(RA, Dec, muRA, sigmuRA, muDec, sigmuDec, radv, sigradv, para, sigpara):
     # Using Formulae from Johnson and SoderBlom 1987
     # Updated values of NGP to fit into the ICRS

     # Takes astrometric parameters in ICRS j2000 and returns the heliocentric velocities in the galactic coordinates,
     # in a right handed (wrt the rotation) sense. Parameters are
     #  RA/Dec: RA/Dec of the object, in decimal degrees
     #  para,sigpara: Parallax and the associated uncertainty, in arcsec
     #  radv,sigradv: Radial Velocity and the associated uncertainty,in km/s
     #  muRA/muDec,sigmuRA/sigmuDec: Proper motion in RA/Dec and the associated uncertainty, in arcsec/yr.

     # The ICRS from the GAIA documentation (DR3 Section 4.1.7 "Transformations of astrometric data and error propagation")
     alphaNGP = 192.85948 * np.pi / 180.0  # radians, because trig functions use those, not degrees
     deltaNGP = 27.12825 * np.pi / 180.0
     theta0 = (90 + 32.93192) * np.pi / 180.0  # The angle in J&S is "Galactic RA", for lack of a better term.

     # RA,Dec in radians
     RA = RA * np.pi / 180.0
     Dec = Dec * np.pi / 180.0

     # Matrix T from paper
     T1 = np.array([[np.cos(theta0), np.sin(theta0), 0],
                    [np.sin(theta0), -np.cos(theta0), 0],
                    [0, 0, 1]])
     T2 = np.array([[-np.sin(deltaNGP), 0, np.cos(deltaNGP)],
                    [0, -1, 0],
                    [np.cos(deltaNGP), 0, np.sin(deltaNGP)]])
     T3 = np.array([[np.cos(alphaNGP), np.sin(alphaNGP), 0],
                    [np.sin(alphaNGP), -np.cos(alphaNGP), 0],
                    [0, 0, 1]])
     T = T1 @ (T2 @ T3)

     # Matrix A from paper
     A = [[np.cos(RA) * np.cos(Dec), -np.sin(RA), -np.cos(RA) * np.sin(Dec)],
          [np.sin(RA) * np.cos(Dec), np.cos(RA), -np.sin(RA) * np.sin(Dec)],
          [np.sin(Dec), 0, np.cos(Dec)]]

     # Give Velocities in a vector
     k = 4.740470464
     B = (T @ A)
     Vels = B @ np.array([radv, k * muRA / para, k * muDec / para])

     # Uncertainties
     C = B ** 2
     term1 = C @ np.array([sigradv ** 2,
                           (k / para) ** 2 * (sigmuRA ** 2 + (muRA * sigpara / para) ** 2),
                           (k / para) ** 2 * (sigmuDec ** 2 + (muDec * sigpara / para) ** 2)])
     term2 = (2 * muRA * muDec * (k ** 2) * (sigpara ** 2) / para ** 4) * np.array([B[0, 1] * B[0, 2], B[1, 1] * B[1, 2], B[2, 1] * B[2, 2]])

     sigVels = np.sqrt(term1 + term2)

     return (np.array([Vels, sigVels]).T).flatten()


## Statistical Analysis
This code was originally written by Figueira et al. (2016) to compliment a Bayesian correlation analysis technique they had designed

They describe he technique in this paper: https://doi.org/10.1007/s11084-016-9490-5

In [ ]:
"""
Written by Pedro Figueira. 
Original version can be found at https://pedrofigueira@bitbucket.org/pedrofigueira/bayesiancorrelation
See https://doi.org/10.1007/s11084-016-9490-5 for a description of this method
"""

"""
Modified by Thomas Wilson in 2024
"""

"""
Modified by George in 2024
The most up to date version may be found on https://github.com/george112n/How-Metallicity-Affects-Exoplanets
See the 'releases' for static and numbered versions
"""

# import string, sys
from numpy import average, std, array, sqrt

from scipy.stats import rankdata, chi2, norm
from scipy.stats import pearsonr, spearmanr

# from pylab import hist, show

from arviz import hdi

def MCMC_model(x_input, y_input, spearmanrank=False, analytical=True):
    """
    MCMC calculation for the BiVariate Normal distribution assumed
        parameters:
        inputfile: x,y tab-separated file with the data
                    -- or --
                   [] for test running (default)
        spearmanrank: flag to toggle pearsonr calculation (False, set by 
                        default) or spearman's rank (True)
        save_posterior: flag to save the posterior distribution ASCII one-column file.
        analytical: flag to perform an analytical calculation of the posterior 
    """    
        
    # if(inputfile==[]):
    #     # print ("Reading test data.")
    #     # #example from http://sumsar.net/blog/2014/03/bayesian-first-aid-pearson-correlation-test/
    #     # x_input, y_input = np.loadtxt("testdata_male.txt", unpack=True)
    # else:
    #     x_input, y_input = np.loadtxt(inputfile, unpack=True)
        
    
    # x_input = np.array(x_input)
    # y_input = np.array(y_input)

    pearsonRAnalysis = pearsonr(x_input, y_input)

    # print("The pearson's coefficient of the data is %.3f with a %.3f 2-sided p-value " % (pearsonRAnalysis[0], pearsonRAnalysis[1]))
    # print("The spearman's rank coefficient of the data is %.3f with a %.3f 2-sided p-value " % (spearmanr(x_input, y_input)[0], spearmanr(x_input, y_input)[1]))
    
    # print("Ranking data")

    if(spearmanrank):
        #rank variable for spearman's rank calculation
        x_input = rankdata(x_input)
        y_input = rankdata(y_input)
    
            
    if(analytical):
        # print("\n"*2)
        # print("-" * 66) 
        # print("Performing analytical calculation...")
        rho_post = calculate_posterior(pearsonRAnalysis[0], len(x_input))
        hdiResults = hdi(rho_post[:], hdi_prob=0.95)
        parameters = array([average(rho_post), std(rho_post), hdiResults[0], hdiResults[1]])
        # print("The mean, std and 95%% HPD are %.3f, %.3f, and [%.3f, %.3f]." % (np.average(rho_post), np.std(rho_post), pm.utils.hpd(rho_post[:], 1.0-0.95)[0], pm.utils.hpd(rho_post[:], 1.0-0.95)[1]))
        # print("The mean, std and 95%% HPD are %.3f, %.3f, and [%.3f, %.3f]." % (parameters[0], parameters[1], parameters[2], parameters[3]))
        # print("The mean over the std (mean/std) is %.3f" % (parameters[0]/parameters[1]))
        # print ("-" * 66)
        # print("\n"*2) 

    # When iterating over a continuum of different ranges this part consumes a lot of time due to file opening. I do not believe it works either.
        
    # with open('rho_summary.csv','w') as file:
    #     file.write('mean,std,2sigma_low_err,2sigma_up_err,3sigma_low_err,3sigma_up_err\n')
    #     file.write(str(parameters[0])+','+str(parameters[1])+','+str(parameters[0]-parameters[2])+','+str(parameters[3]-parameters[0])+','+str(parameters[0]-az.hdi(rho_post[:], hdi_prob=0.997)[0])+','+str(az.hdi(rho_post[:], hdi_prob=0.997)[1]-parameters[0]))
    
    # Returns some statistics of the distribution and some 
    return parameters, rho_post 

def calculate_posterior(r, n, a=1, b=2, N=10000):
    # function for Analytical posterior calculation

    Z = norm.rvs(size=N)
    chi2_1 = chi2.rvs(n-a, size=N)
    chi2_2 = chi2.rvs(n-b, size=N)
    
    sqrtchi2_1 = sqrt(chi2_1)

    Y = - Z/sqrtchi2_1 + (sqrt(chi2_2) / sqrtchi2_1) * (r / sqrt(1-r**2))
    rho_post = Y / sqrt(1+Y**2.0)
    return rho_post
    

## Input analysis parametres here
Set up the variables here, including file name, and plot data and axes

In [ ]:
# Determines whether the file name is to take user input or take the hard coded value
bCustomFileName = False
szFileName = "PS_2024.02.19_11.15.47.csv" # Only use if above is set to false

szGraphTitle = ""

#The file name of the graph. Must end in .svg
saveFileName = "Planet Density-Stellar Metallicity.png"

if bCustomFileName:
    # Accepts user input of the file name
    szFileName = input("Input the file name of the exoplanet data: ")

## Reads exoplanet data from file
Omits controversial data and rows which are not the default parameters for the given planet.

In [ ]:
print("Loading exoplanet data from file '", szFileName, "'")

# Reads the exoplanet data from the file
allData = readData(szFileName)

# Omit data with a Controversial flag (pl_controv_flag)
allNonControversialData = allData[allData['pl_controv_flag'] == 0]

# Confrain the planets to planets with the default_flag (Avoids repeated plotting) (default_flag)
allNonControversialAndDefaultData = allNonControversialData[allNonControversialData['default_flag'] == 1]

## Analysing planet properties againt their host star properties
### Compiling the list of planets

In [ ]:
# ------------------------------------------------------------------------
# --------------------- Planet property quality cuts ---------------------
# ------------------------------------------------------------------------

# Perform planet property quality cuts before GAIA query in order to reduce the amount of network data
allGoodExoplanets = allNonControversialData[allNonControversialData['default_flag'] == 1]

# Quality cut with the insolation data
allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_insollim'] == 0]
allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_insolerr1']/allGoodExoplanets['pl_insol'] < 0.2]
allGoodExoplanets = allGoodExoplanets[(allGoodExoplanets['pl_insolerr2']*-1)/allGoodExoplanets['pl_insol'] < 0.2]

# Only accept planets with a defined density
# allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_denslim'] != 1]
# allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_denserr1']/allGoodExoplanets['pl_dens'] < 0.2]
# allGoodExoplanets = allGoodExoplanets[(allGoodExoplanets['pl_denserr2']*-1)/allGoodExoplanets['pl_dens'] < 0.2]

# Quality cut with the mass data
allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_masselim'] == 0]
allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_masseerr1']/allGoodExoplanets['pl_masse'] < 0.2]
allGoodExoplanets = allGoodExoplanets[(allGoodExoplanets['pl_masseerr2']*-1)/allGoodExoplanets['pl_masse'] < 0.2]

# Add the colour dimension (for fun)
# allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_insol'] > -1]

# ---------- Radii ---------- #
allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_radelim'] == 0]
allGoodExoplanets = allGoodExoplanets[allGoodExoplanets['pl_radeerr1']/allGoodExoplanets['pl_rade'] < 0.2]
allGoodExoplanets = allGoodExoplanets[(allGoodExoplanets['pl_radeerr2']*-1)/allGoodExoplanets['pl_rade'] < 0.2]

#print(allGoodExoplanets)
# print(allGoodExoplanets.to_string())

# Extract the coordinates and gaia magnitudes of all of the NASA exoplanet hosting stars

# Create astropy table from NASA data of exoplanet hosting stars
allGoodExoplanetsTable = QTable()
allGoodExoplanetsTable['Density'] = allGoodExoplanets['pl_dens'].to_numpy() * (units.g/(units.cm)**3)
allGoodExoplanetsTable['Density Error Up'] = allGoodExoplanets['pl_denserr1'].to_numpy() * (units.g/(units.cm)**3)
allGoodExoplanetsTable['Density Error Down'] = allGoodExoplanets['pl_denserr2'].to_numpy() * (units.g/(units.cm)**3)
allGoodExoplanetsTable['Mass (Earth)'] = allGoodExoplanets['pl_masse'].to_numpy()
allGoodExoplanetsTable['Mass (Earth) Error Up'] = allGoodExoplanets['pl_masseerr1'].to_numpy()
allGoodExoplanetsTable['Mass (Earth) Error Down'] = allGoodExoplanets['pl_masseerr2'].to_numpy()
allGoodExoplanetsTable['Radius (Earth)'] = allGoodExoplanets['pl_rade'].to_numpy()
allGoodExoplanetsTable['Radius (Earth) Error Up'] = allGoodExoplanets['pl_radeerr1'].to_numpy()
allGoodExoplanetsTable['Radius (Earth) Error Down'] = allGoodExoplanets['pl_radeerr2'].to_numpy()
allGoodExoplanetsTable['Right Ascension'] = allGoodExoplanets["ra"].to_numpy() * units.deg
allGoodExoplanetsTable['Declination'] = allGoodExoplanets["dec"].to_numpy() * units.deg
allGoodExoplanetsTable['GAIA Magnitude'] = allGoodExoplanets["sy_gaiamag"].to_numpy()
allGoodExoplanetsTable['Insolation'] = allGoodExoplanets['pl_insol'].to_numpy()

# Filter out exoplanet hosting stars which don't have a GAIA magnitude
allGoodExoplanetsTable = allGoodExoplanetsTable[(np.isnan(allGoodExoplanetsTable['GAIA Magnitude']) == False)]

# Filter out exoplanet hosting stars which don't have a radius or mass or density
allGoodExoplanetsTable = allGoodExoplanetsTable[(np.isnan(allGoodExoplanetsTable['Radius (Earth)']) == False)]
allGoodExoplanetsTable = allGoodExoplanetsTable[(np.isnan(allGoodExoplanetsTable['Mass (Earth)']) == False)]
# allGoodExoplanetsTable = allGoodExoplanetsTable[(np.isnan(allGoodExoplanetsTable['Density']) == False)]
allGoodExoplanetsTable = allGoodExoplanetsTable[(np.isnan(allGoodExoplanetsTable['Insolation']) == False)]

print("Amount of exoplanet hosting stars relevant to my search (includes duplicates purposefully ofc): "+str(len(allGoodExoplanetsTable)) +"\n")



# -------------------------------------------------------------------------
# -------------------------- Get star properties --------------------------
# -------------------------------------------------------------------------

# Initialise GAIA star table
gaiaExoplanetStars = QTable()

# Query GAIA for each of these exoplanet hosting stars - create a table of all of the exoplanet hosting stars and their properties
gaiaExoplanetStars = getGAIADataMultipleStars2(allGoodExoplanetsTable)

# Print the stars returned from the query
# print("All selected stars from GAIA archive: (Total = ", len(gaiaExoplanetStars), ")")
# print("Total:", len(gaiaExoplanetStars))
# gaiaExoplanetStars.pprint(max_width=50000, max_lines=5000)

# print("All selected stars from the GAIA archive")
print("All selected stars from the GAIA archive:", len(gaiaExoplanetStars))

# Create a copy of the star data but only for ones with metallicity values - Filter out stars with no alphafe_gspspec metallicity or Fe metallicity
gaiaExoplanetStarsWithMetallicities = gaiaExoplanetStars[(gaiaExoplanetStars['mh_gspphot'].mask == False)]
# gaiaExoplanetStarsWithMetallicities = gaiaExoplanetStarsWithMetallicities[(gaiaExoplanetStarsWithMetallicities['alphafe_gspspec'].mask == False)]

print("All selected stars from the GAIA archive with Fe/H values")
print("Total:", len(gaiaExoplanetStarsWithMetallicities))
# gaiaExoplanetStarsWithMetallicities.pprint(max_width=50000, max_lines=5000)

# Go through all of the exoplanets in the table and add star details. Compiles a list of planets with star data

# ------------------------------------------------------------------------
# --------------------- Match planets to their stars ---------------------
# ------------------------------------------------------------------------
print("Matching planets to their stars")

print("There are", len(allGoodExoplanetsTable), "planets")
allGoodExoplanetsWithStarData = QTable()
bFirstRowAdded = False

# Box width for filtering
width=8*units.arcsecond

# Go through all of the exoplanets
for i in range(len(allGoodExoplanetsTable)):
    print("Planet", i, "| GAIA magnitude =", allGoodExoplanetsTable[i]['GAIA Magnitude'])
    # Go through all of the stars and find the correct one for the planet
    for j in range(len(gaiaExoplanetStarsWithMetallicities)):
        # If this star is the one for the planet, add the stars details to the planet
        if (gaiaExoplanetStarsWithMetallicities[j]['phot_g_mean_mag'] <= allGoodExoplanetsTable[i]['GAIA Magnitude'] + 0.5 and
            gaiaExoplanetStarsWithMetallicities[j]['phot_g_mean_mag'] >= allGoodExoplanetsTable[i]['GAIA Magnitude'] - 0.5 and
            gaiaExoplanetStarsWithMetallicities[j][GaiaClass.MAIN_GAIA_TABLE_RA] <= (allGoodExoplanetsTable[i]['Right Ascension'] + width).value and
            gaiaExoplanetStarsWithMetallicities[j][GaiaClass.MAIN_GAIA_TABLE_RA] >= (allGoodExoplanetsTable[i]['Right Ascension'] - width).value and 
            gaiaExoplanetStarsWithMetallicities[j][GaiaClass.MAIN_GAIA_TABLE_DEC] <= (allGoodExoplanetsTable[i]['Declination'] + width).value and
            gaiaExoplanetStarsWithMetallicities[j][GaiaClass.MAIN_GAIA_TABLE_DEC] >= (allGoodExoplanetsTable[i]['Declination'] - width).value):
            
            # Combines the exoplanet's own data with its star's data
            hostStarData = gaiaExoplanetStarsWithMetallicities[j]
            exoplanetAndHostStar = hstack([hostStarData, allGoodExoplanetsTable[i]])

            # And add to the list of planets
            if (bFirstRowAdded):
                allGoodExoplanetsWithStarData = vstack([allGoodExoplanetsWithStarData, exoplanetAndHostStar])
                print("Star identified")
            else:
                allGoodExoplanetsWithStarData = exoplanetAndHostStar
                bFirstRowAdded = True
                print("Star identified")
            
            # Stops the inner for loop if a star for that planet was found - if there were 2 matches (which there would be in some instances because some stars are duplicated) then it just takes the first one as the correct one for the planet
            break

print("There are", len(allGoodExoplanetsWithStarData), "planets with star data")

# Can now plot the planets with their star data, from allGoodExoplanetsWithStarData
# Or filter, or calculate errors etc


### Filtering, adding new columns, etc

In [ ]:
# -------------------------------------------------------------------------
# --------------- Deal with the data, analyse, filter, etc. ---------------
# -------------------------------------------------------------------------

# Calculate Fe/H errors, and filter
allGoodExoplanetsWithStarData['FeHErrorsUp'] = allGoodExoplanetsWithStarData['mh_gspphot_upper'] - allGoodExoplanetsWithStarData['mh_gspphot']
allGoodExoplanetsWithStarData['FeHErrorsLow'] = allGoodExoplanetsWithStarData['mh_gspphot'] - allGoodExoplanetsWithStarData['mh_gspphot_lower']
allGoodExoplanetsWithStarData = allGoodExoplanetsWithStarData[allGoodExoplanetsWithStarData['FeHErrorsUp'] < 0.1]
allGoodExoplanetsWithStarData = allGoodExoplanetsWithStarData[allGoodExoplanetsWithStarData['FeHErrorsLow'] < 0.1]


# Calculate Alpha/Fe errors
allGoodExoplanetsWithStarData['AlphaFeErrorsUp'] = allGoodExoplanetsWithStarData['alphafe_gspspec_upper'] - allGoodExoplanetsWithStarData['alphafe_gspspec']
allGoodExoplanetsWithStarData['AlphaFeErrorsLow'] = allGoodExoplanetsWithStarData['alphafe_gspspec'] - allGoodExoplanetsWithStarData['alphafe_gspspec_lower']

# Create the new columns for galactic velocities
allGoodExoplanetsWithStarData['U'] = np.empty((len(allGoodExoplanetsWithStarData)))
allGoodExoplanetsWithStarData['UErr'] = np.empty((len(allGoodExoplanetsWithStarData)))
allGoodExoplanetsWithStarData['V'] = np.empty((len(allGoodExoplanetsWithStarData)))
allGoodExoplanetsWithStarData['VErr'] = np.empty((len(allGoodExoplanetsWithStarData)))
allGoodExoplanetsWithStarData['W'] = np.empty((len(allGoodExoplanetsWithStarData)))
allGoodExoplanetsWithStarData['WErr'] = np.empty((len(allGoodExoplanetsWithStarData)))
allGoodExoplanetsWithStarData['Speed'] = np.empty((len(allGoodExoplanetsWithStarData)))
allGoodExoplanetsWithStarData['SpeedErr'] = np.empty((len(allGoodExoplanetsWithStarData)))

# Go through all stars, calculate galactic velocities
print("Calculating galactic velocities")
for i in range (len(allGoodExoplanetsWithStarData)):
    gaiaStar = allGoodExoplanetsWithStarData[i]
    
    galacticVelocity = computeGalacticVelocity(gaiaStar)
    allGoodExoplanetsWithStarData['U'][i] = galacticVelocity[0]
    allGoodExoplanetsWithStarData['UErr'][i] = galacticVelocity[1]
    allGoodExoplanetsWithStarData['V'][i] = galacticVelocity[2]
    allGoodExoplanetsWithStarData['VErr'][i] = galacticVelocity[3]
    allGoodExoplanetsWithStarData['W'][i] = galacticVelocity[4]
    allGoodExoplanetsWithStarData['WErr'][i] = galacticVelocity[5]
    allGoodExoplanetsWithStarData['Speed'][i] = (galacticVelocity[0]**2 + galacticVelocity[2]**2 + galacticVelocity[4]**2)**(0.5)
    allGoodExoplanetsWithStarData['SpeedErr'][i] = (galacticVelocity[1]**2 + galacticVelocity[3]**2 + galacticVelocity[5]**2)**(0.5) # Is this even correct

# Print number of final stars
print("There are now", len(allGoodExoplanetsWithStarData), "planets with star data")

# Start plotting the data in various ways

planetDensity = allGoodExoplanetsWithStarData['Density']
planetDensityLabel = "Planet Density (g/cm$^{3}$)"
planetDensityError = np.stack((allGoodExoplanetsWithStarData['Density Error Up'].value, -1 * allGoodExoplanetsWithStarData['Density Error Down'].value))

planetMass = allGoodExoplanetsWithStarData['Mass (Earth)']
planetMassLabel = "Planet Mass (M$_{\oplus}$)"
planetMassError = np.stack((allGoodExoplanetsWithStarData['Mass (Earth) Error Up'].value, -1 * allGoodExoplanetsWithStarData['Mass (Earth) Error Down'].value))

planetRadius = allGoodExoplanetsWithStarData['Radius (Earth)']
planetRadiusLabel = "Planet Radius (R$_{\oplus}$)"
planetRadiusError = np.stack((allGoodExoplanetsWithStarData['Radius (Earth) Error Up'].value, -1 * allGoodExoplanetsWithStarData['Radius (Earth) Error Down'].value))

#yError = galacticSpeeds.T[1]
#fullYError = np.stack((yError, yError))

FeHMetallicity = allGoodExoplanetsWithStarData['mh_gspphot'].value
FeHMetalicityLabel = "[Fe/H] Metallicity (Dex)"
FeHMetallicityError = np.stack((allGoodExoplanetsWithStarData['FeHErrorsLow'].value, allGoodExoplanetsWithStarData['FeHErrorsUp'].value))

AlphaFeMetallicity = allGoodExoplanetsWithStarData['alphafe_gspspec'].value
AlphaFeMetalicityLabel = "[Alpha/Fe] Metallicity (Dex)"
AlphaFeMetallicityError = np.stack((allGoodExoplanetsWithStarData['AlphaFeErrorsLow'].value, allGoodExoplanetsWithStarData['AlphaFeErrorsUp'].value))

U = allGoodExoplanetsWithStarData['U'].value
UErr = np.stack((allGoodExoplanetsWithStarData['UErr'].value, allGoodExoplanetsWithStarData['UErr'].value))
V = allGoodExoplanetsWithStarData['V'].value
VErr = np.stack((allGoodExoplanetsWithStarData['VErr'].value, allGoodExoplanetsWithStarData['VErr'].value))
W = allGoodExoplanetsWithStarData['W'].value
WErr = np.stack((allGoodExoplanetsWithStarData['WErr'].value, allGoodExoplanetsWithStarData['WErr'].value))
Speed = allGoodExoplanetsWithStarData['Speed'].value
SpeedErr = np.stack((allGoodExoplanetsWithStarData['SpeedErr'].value, allGoodExoplanetsWithStarData['SpeedErr'].value))

# Plots

In [ ]:
# allGoodExoplanetsWithStarData.pprint_all()

print("Size of planet and star data:", str(len(allGoodExoplanetsWithStarData)))
print("Size of metallicities:", str(len(FeHMetallicity)))

# Make copies of it and each copy filter a different range:
# Needs to be before the referencing. Well, since the referencing will need to be added to the loop anyway we might as well delete the rereferencing, or at least just reference the table directly for the for loop ones


planetsAndTheirHostsRadiiRange = allGoodExoplanetsWithStarData[allGoodExoplanetsWithStarData['Insolation'] >= 250]


# For each radius range
for i in range(1):
    i = 2
    if (i == 0):
        planetsAndTheirHostsRadiiRange = planetsAndTheirHostsRadiiRange[planetsAndTheirHostsRadiiRange['Radius (Earth)'] <= 3]
    elif (i == 1):
        planetsAndTheirHostsRadiiRange = planetsAndTheirHostsRadiiRange[planetsAndTheirHostsRadiiRange['Mass (Earth)'] >= 130]
        planetsAndTheirHostsRadiiRange = planetsAndTheirHostsRadiiRange[planetsAndTheirHostsRadiiRange['Mass (Earth)'] <= 520]
        # planetsAndTheirHostsRadiiRange = planetsAndTheirHostsRadiiRange[planetsAndTheirHostsRadiiRange['Mass (Earth)'] >= 300]
        # planetsAndTheirHostsRadiiRange = planetsAndTheirHostsRadiiRange[planetsAndTheirHostsRadiiRange['Density'] <= 1]
    elif (i == 2):
        planetsAndTheirHostsRadiiRange = allGoodExoplanetsWithStarData
    
    
    # For each of the planet parameters
    # for j in range(1):
    szParameter = "mass"
    if (szParameter == "radius"):
        yData = planetsAndTheirHostsRadiiRange['Radius (Earth)']
        yLabel = planetRadiusLabel
        yError = np.stack((planetsAndTheirHostsRadiiRange['Radius (Earth) Error Up'].value, -1 * planetsAndTheirHostsRadiiRange['Radius (Earth) Error Down'].value))
        logariseY = False
    
    elif (szParameter == "mass"):
        yData = planetsAndTheirHostsRadiiRange['Mass (Earth)']
        yLabel = planetMassLabel
        yError = np.stack((planetsAndTheirHostsRadiiRange['Mass (Earth) Error Up'].value, -1 * planetsAndTheirHostsRadiiRange['Mass (Earth) Error Down'].value))
        logariseY = True
    
    elif (szParameter == "density"):
        yData = planetsAndTheirHostsRadiiRange['Density'].value
        yLabel = planetDensityLabel
        yError = np.stack((planetsAndTheirHostsRadiiRange['Density Error Up'].value, -1 * planetsAndTheirHostsRadiiRange['Density Error Down'].value))
        logariseY = True

    # For each of the star parameters
    # for k in range(2):
    k = 0
    if (k == 0):
        xData = planetsAndTheirHostsRadiiRange['mh_gspphot'].value
        xLabel = FeHMetalicityLabel
        xError = np.stack((planetsAndTheirHostsRadiiRange['FeHErrorsLow'].value, planetsAndTheirHostsRadiiRange['FeHErrorsUp'].value))
    # elif (k == 1):
    #     xData = planetsAndTheirHostsRadiiRange['alphafe_gspspec'].value
    #     xLabel = "[Alpha/Fe] Metallicity (Dex)"
    #     xError = np.stack((planetsAndTheirHostsRadiiRange['AlphaFeErrorsLow'].value, planetsAndTheirHostsRadiiRange['AlphaFeErrorsUp'].value))
    # elif (k == 2):
    #     xData = planetsAndTheirHostsRadiiRange['U'].value
    #     xLabel = "U [km/s]"
    #     xError = np.stack((planetsAndTheirHostsRadiiRange['UErr'].value, planetsAndTheirHostsRadiiRange['UErr'].value))
    # elif (k == 3):
    #     xData = planetsAndTheirHostsRadiiRange['V'].value
    #     xLabel = "V [km/s]"
    #     xError = np.stack((planetsAndTheirHostsRadiiRange['VErr'].value, planetsAndTheirHostsRadiiRange['VErr'].value))
    # elif (k == 4):
    #     xData = planetsAndTheirHostsRadiiRange['W'].value
    #     xLabel = "W [km/s]"
    #     xError = np.stack((planetsAndTheirHostsRadiiRange['WErr'].value, planetsAndTheirHostsRadiiRange['WErr'].value))
    # else:
    #     xData = planetsAndTheirHostsRadiiRange['Speed'].value
    #     xLabel = "Speed [km/s]"
    #     xError = np.stack((planetsAndTheirHostsRadiiRange['SpeedErr'].value, planetsAndTheirHostsRadiiRange['SpeedErr'].value))

    # Creates a scatter graph
    # if i==0:
    #     szGraphTitle = (yLabel +" vs " +xLabel +" with radius <= 3 Re")
    # elif i==1:
    #     szGraphTitle = (yLabel +" vs " +xLabel +" with density between 0 and 1.5 g/cm^3")
    #     # planetsAndTheirHostsRadiiRange.pprint()
    # elif i==2:
    #     szGraphTitle = (yLabel +" vs " +xLabel +" with $x$ density >= 2.25 g/cm^3")
    #     szGraphTitle = (yLabel +" vs " +xLabel +" with errors less than 20%")


    # Edit this
    szColourAxis = "insolation"
    bColourLoggersised = True

    if (szColourAxis == "mass"):
        colourData = planetsAndTheirHostsRadiiRange['Mass (Earth)'].value
        colourLabel = planetMassLabel
    if (szColourAxis == "radius"):
        colourData = planetsAndTheirHostsRadiiRange['Radius (Earth)'].value
        colourLabel = planetRadiusLabel
    if (szColourAxis == "density"):
        colourData = planetsAndTheirHostsRadiiRange['Density'].value
        colourLabel = planetDensityLabel
    if (szColourAxis == "insolation"):
        colourData = planetsAndTheirHostsRadiiRange['Insolation'].value
        colourLabel = "Insolation [Insol Units]"
    
    
    # CBA to go through and comment them all out
    szGraphTitle = ""

    logariseY = False
    
    # plt.show()
    # print(colourData)
        
    figure = plotScatterGraphOfData(szTitle="", xAxis=xLabel, yAxis=yLabel, xData=xData, yData=yData, xError=xError, yError=yError,
                                    logariseXAxis=False, logariseYAxis=logariseY, colours=colourData, colourAxisLabel=colourLabel, bColourLoggersised=bColourLoggersised, 
                                    sizes=planetsAndTheirHostsRadiiRange['Radius (Earth)']*0.2, tickSeperation = 0.5)

    # if (i==2 and j == 0):
    #     axes.hlines(y = [0.5, 10.7, 5.7, 19.3, 2.65, 4.85], colors=['b', 'b', 'r', 'r', 'g', 'g'], linestyles='dashed', xmin=-1.4, xmax=0.9)
    
    bbox = Bbox.from_bounds(0, 0, 3.2, 2.4)
    bbox = Bbox.from_bounds(0.26, -0.2, *bbox.size)
    tightbbox = figure.get_tightbbox(figure.canvas.get_renderer())
    print(tightbbox)
    # just set it directly to figure size
    figure.set(figheight=tightbbox.height, figwidth=tightbbox.width)
    print(figure.get_figwidth())
    
    figure.savefig(fname = "Graphs/s.pdf", format="pdf", bbox_inches=bbox)
    figure.show()

    # saveGraph("Exported graph", "pdf")

    parameters, rho_post = MCMC_model(xData, yData)
    
    szLine = "%ld %.3lf %.3lf %.3lf %.3lf %.3lf" % (len(planetsAndTheirHostsRadiiRange), parameters[0], parameters[1], parameters[2], parameters[3], (parameters[0]/parameters[1]))
    print(szLine)
    plt.figure(dpi=100)
    plt.hist(rho_post, range = (-0.8, 0.8), bins=100)
    plt.title(szGraphTitle)

print(np.average(planetsAndTheirHostsRadiiRange['Density'].value))



## Analysis of the switch point for radii for radii against Fe/H
#### The range R > will be varied and the curve plotted.

In [ ]:
# Defines the start value for the lower limit. Is increased until lowerUpperLimit
currentLowerLimit = 0

# Defines the bounds of the correlation space
lowerUpperLimit = 25
upperUpperLimit = 25
newLength = 0


def stepUp(currentLimit):
    # # Masses
    # if (currentLimit < 10):
    #     limitMove = 0.04
    # elif (currentLimit < 50):
    #     limitMove = 0.04
    # elif (currentLimit < 200):
    #     limitMove = 0.4
    # elif (currentLimit < 5000):
    #     limitMove = 40
    # else:
    #     limitMove = 400

    # Radii
    if (currentLimit < 15):
        limitMove = 0.01
    else:
        limitMove = 0.1

    # Densities (Already been modified to increase resolution) - this was without the 6 cores though, so I'm going to increase resolution further
    # if (currentLimit < 4.99):
    #     limitMove = 0.002
    # elif (currentLimit < 10.99):
    #     limitMove = 0.02
    # elif (currentLimit < 14.99):
    #     limitMove = 0.04
    # else:
    #     limitMove = 1

    # 900,000 with radii
    # if (currentLimit < 4.99):
    #     limitMove = 0.004
    # elif (currentLimit < 10.99):
    #     limitMove = 0.05
    # elif (currentLimit < 14.99):
    #     limitMove = 0.08
    # else:
    #     limitMove = 1

    # if (currentLimit < 4.99):
    #     limitMove = 0.005
    # elif (currentLimit < 10.99):
    #     limitMove = 0.1
    # elif (currentLimit < 14.99):
    #     limitMove = 0.2
    # else:
    #     limitMove = 1
    
    
    return limitMove


# Loops over the upper limit
def innerLoop(currentLowerLimit):
    # Start the upper limit at the top values, then slowly decrease it 
    currentUpperLimit = upperUpperLimit

    #Resets the list to include everything above the lower limit
    planetsAndTheirHostsLowerPlanetsExcluded = allGoodExoplanetsWithStarData[allGoodExoplanetsWithStarData['Radius (Earth)'] >= currentLowerLimit]
    # Stores the content of one inner loop (so multiple lines) before writting them all in one go
    szOneInnerLoop= ""

    while currentUpperLimit > currentLowerLimit: # Iterate from upperUpperLimit to currentLowerLimit
        # Restrticts the list to be everything under the upper limit
        planetsAndTheirHostsRadiiRange = planetsAndTheirHostsLowerPlanetsExcluded[planetsAndTheirHostsLowerPlanetsExcluded['Radius (Earth)'] <= currentUpperLimit]
        
        if (len(planetsAndTheirHostsRadiiRange) < 8): # Fck it because we really don't use these 5 ones
            currentUpperLimit = currentUpperLimit - stepUp(currentUpperLimit)
            # print("Not enough planets in range", currentLowerLimit, upperLowerLimit)
            # continue
            break # we can now break because we know any ranges below this will also not have 5 planets since we only reduce the range within this inner loop
        
        # if (newLength != lengthStore):
        parameters, rho_post = MCMC_model(planetsAndTheirHostsRadiiRange['mh_gspphot'].value, planetsAndTheirHostsRadiiRange['Mass (Earth)'].value)
        
        szLine = "%.2lf %.2lf %ld %.3lf %.3lf %.3lf %.3lf %.3lf" % (currentLowerLimit, currentUpperLimit, len(planetsAndTheirHostsRadiiRange), parameters[0], parameters[1], parameters[2], parameters[3], (parameters[0]/parameters[1]))
        # print(szLine)
        szOneInnerLoop = "\n".join([szOneInnerLoop, szLine]) 
        # print("The mean over the std (mean/std) is %.3f" % (parameters[0]/parameters[1]))
        # np.savetxt("rho_summary.csv", rho_post, fmt="%d", delimiter=",") #saving the results on .csv format
        currentUpperLimit = currentUpperLimit - stepUp(currentUpperLimit)
        # lengthStore = newLength
        # plt.figure(dpi=100)
        # plt.hist(rho_post, range = (-0.8, 0.8), bins=80)
        # plt.title(("Radius vs Fe/H for radius >= 1.2 and <=" + str(upperLimit).format("%.2lf") + "Re"))
        # plt.show()
    
    szOutputMessage = "Loop for lower limit = " + ("%.3lf " % (currentLowerLimit)) + "finished"
    print(szOutputMessage)
    file = open("FeH vs Mass for all Radius ranges (more than 250 Insolation).txt", "a")
    file.write(szOneInnerLoop)
    file.close()



### The actual runthrough

In [ ]:
# from multiprocessing import Pool
# from multiprocess import Pool
from multiprocess import Process

#This will use six cores. Add or remove to use more.
while currentLowerLimit <= lowerUpperLimit:
    process1 = Process(target=innerLoop, args=[currentLowerLimit])
    currentLowerLimit = currentLowerLimit + stepUp(currentLowerLimit)
    process1.start()

    process2 = Process(target=innerLoop, args=[currentLowerLimit])
    currentLowerLimit = currentLowerLimit + stepUp(currentLowerLimit)
    process2.start()

    process3 = Process(target=innerLoop, args=[currentLowerLimit])
    currentLowerLimit = currentLowerLimit + stepUp(currentLowerLimit)
    process3.start()

    process4 = Process(target=innerLoop, args=[currentLowerLimit])
    currentLowerLimit = currentLowerLimit + stepUp(currentLowerLimit)
    process4.start()

    process5 = Process(target=innerLoop, args=[currentLowerLimit])
    currentLowerLimit = currentLowerLimit + stepUp(currentLowerLimit)
    process5.start()

    process6 = Process(target=innerLoop, args=[currentLowerLimit])
    currentLowerLimit = currentLowerLimit + stepUp(currentLowerLimit)
    process6.start()

    process1.join()
    process2.join()
    process3.join()
    process4.join()
    process5.join()
    process6.join()


# #Create all processes in one go
# # create the process pool with 4 cores
# allLowerLimits = np.array([currentLowerLimit])
# while currentLowerLimit <= lowerUpperLimit:
#     currentLowerLimit = currentLowerLimit + stepUp(currentLowerLimit)
#     allLowerLimits = np.append(allLowerLimits, [currentLowerLimit])

# if __name__ == '__main__':
#     thePool = Pool(4)

#     thePool.map_async(func=innerLoop, iterable=allLowerLimits)

#     thePool.join()


## Peforms the analysis steps for Planet-Planet Properties
This can be edited as you see fit. If extra features need to be added to a graph, do it here

In [ ]:
# ------------------------------------------------------------------------
# --------------------- Planet property quality cuts ---------------------
# ------------------------------------------------------------------------

allGraphData = allNonControversialAndDefaultData[allNonControversialAndDefaultData['pl_denslim'] != 1]
allGraphData = allGraphData[allGraphData['pl_masselim'] != 1]
allGraphData = allGraphData[allGraphData['st_metlim'] != 1]
allGraphData = allGraphData[allGraphData['pl_insollim'] != 1]
allGraphData = allGraphData[allGraphData['st_metratio'] == "[Fe/H]"]
# allGraphData = allGraphData[allGraphData['pl_dens'] < 8]
allGraphData = allGraphData[allGraphData['pl_masse'] <= 20]
allGraphData = allGraphData[allGraphData['pl_insol'] > -1]

# Remove uncertain data
allGraphData = allGraphData[allGraphData['pl_masseerr1']/allGraphData['pl_masse'] < 0.2]
allGraphData = allGraphData[allGraphData['pl_masseerr2']/allGraphData['pl_masse'] < 0.2]

allGraphData = allGraphData[allGraphData['st_meterr1'] < 0.2]
allGraphData = allGraphData[allGraphData['st_meterr2'] < 0.2]

# allGraphData = allGraphData[allGraphData['pl_denserr1']/allGraphData['pl_dens'] < 0.2]
# allGraphData = allGraphData[allGraphData['pl_denserr2']/allGraphData['pl_dens'] < 0.2]

# Create a colour dictionary of detection methods
# colours = detectionMethodColourDictionary(allGraphData)

# ------------------------------------------------------------------------
# ----------------------- Planet property plotting -----------------------
# ------------------------------------------------------------------------

# Specify the axes
yLabel = "Planet Mass [Earth Masses]"
yField = "pl_masse"

xLabel = "Stellar Metalilcity [dex]"
xField = "st_met"
# yLabel = "Planet Density"
# yField = "pl_dens"

# Add the colour dimension
colours = allGraphData['pl_insol']
colourAxisLabel = "Planet Insolation"

# Extracts the errors - Yes, down is 2 and up is 1
xErrorDown = (allGraphData[xField+"err2"] * -1).to_numpy()
xErrorUp = allGraphData[xField+"err1"].to_numpy()
xError = np.stack((xErrorDown, xErrorUp))

yErrorDown = (allGraphData[yField+"err2"] * -1).to_numpy()
yErrorUp = allGraphData[yField+"err1"].to_numpy()
yError = np.stack((yErrorDown, yErrorUp))


# Creates a scatter graph
plotScatterGraphOfData(szGraphTitle, xLabel, yLabel, allGraphData[xField], allGraphData[yField], xError, yError, False, True,
                      colours=colours.to_numpy(), colourAxisLabel = colourAxisLabel)

# iMax = 0
# iMin = 99999999999
# for i in colours:
#     if i < iMin:
#         iMin = i
#     if i > iMax:
#         iMax = i

# print(iMax)
# print(iMin)


#plt.legend("Discovery Methods")
#plt.legend("Red = Radial Velocity")
#plt.legend("Blue = Transit")
#plt.legend("Green = Imaging")
#plt.legend("Magenta = Other")

# Saves the graph
# saveGraph(saveFileName, "png")
# plt.show()


###########Analsyis 2 - the planet boi one

        # This is here because we only want a list of densities for the planets where the stars metallicity data is good quality
        # Density quality was ensured before the query was even sent so that is already intrinsic to this data
    #    PlanetDensitiesErrorsUp = np.append(PlanetDensitiesErrorsUp, allGoodExoplanets['pl_denserr1'].iloc[i])
    #    PlanetDensitiesErrorsLow = np.append(PlanetDensitiesErrorsLow, allGoodExoplanets['pl_denserr2'].iloc[i]*-1)
    #    PlanetDensities = np.append(PlanetDensities, allGoodExoplanets['pl_dens'].iloc[i])
#
    #    colourData = np.append(colourData, allGoodExoplanetsTable['masse'][i])



## Request and download GAIA data for exoplanet hosting stars

In [ ]:
# Set up the exoplanets
allGoodExoplanets = allNonControversialAndDefaultData

# Extract the coordinates and gaia magnitudes of all of the NASA exoplanet hosting stars
RAs = allGoodExoplanets["ra"]
Decs = allGoodExoplanets["dec"]
gaiaMagnitudesNasa = allGoodExoplanets["sy_gaiamag"]

# Remove duplicate stars
index = -1
for i in range (len(RAs) - 1):
    index = index + 1
    if RAs.iloc[index+1] == RAs.iloc[index]:
        if Decs.iloc[index+1] == Decs.iloc[index]:
            if gaiaMagnitudesNasa.iloc[index+1] == gaiaMagnitudesNasa.iloc[index]:
                szLabel = RAs.index[index+1]
                RAs.drop(labels=szLabel, axis="index", inplace=True)
                Decs.drop(labels=szLabel, axis="index", inplace=True)
                gaiaMagnitudesNasa.drop(labels=szLabel, axis="index", inplace=True)
                index = index - 1

print("Total exoplanet hosting stars:", len(RAs))

# Create astropy table from NASA data of exoplanet hosting stars
coordsAndMagnitude = QTable()
coordsAndMagnitude['Right Ascension'] = RAs.to_numpy() * units.deg
coordsAndMagnitude['Declination'] = Decs.to_numpy() * units.deg
coordsAndMagnitude['GAIA Magnitude'] = gaiaMagnitudesNasa.to_numpy()

#coordsAndMagnitude.pprint(max_width=50000, max_lines=5000)

# Filter out exoplanet hosting stars which don't have a GAIA magnitude
coordsAndMagnitude = coordsAndMagnitude[(np.isnan(coordsAndMagnitude['GAIA Magnitude']) == False)]

print("Total exoplanet hosting stars with GAIA magnitude:", len(coordsAndMagnitude['GAIA Magnitude']))
# coordsAndMagnitude.pprint(max_width=50000, max_lines=5000)

# Initialise GAIA star table
gaiaExoplanetStars = QTable()

# Query GAIA for each of these exoplanet hosting stars - create a table of all of the exoplanet hosting stars and their properties
gaiaExoplanetStars = getGAIADataMultipleStars2(coordsAndMagnitude)

# Print the stars returned from the query
print("All selected stars from GAIA archive: (Total = ", len(gaiaExoplanetStars), ")")
print("Total:", len(gaiaExoplanetStars))
gaiaExoplanetStars.pprint(max_width=50000, max_lines=5000)

# Create a copy of the star data but only for ones with metallicity values -
# (Filter out stars with no alphafe_gspspec metallicity or Fe metallicity)
gaiaExoplanetStarsWithMetallicities = gaiaExoplanetStars[(gaiaExoplanetStars['alphafe_gspspec'].mask == False)]
gaiaExoplanetStarsWithMetallicities = gaiaExoplanetStarsWithMetallicities[(gaiaExoplanetStarsWithMetallicities['mh_gspphot'].mask == False)]

print("All selected stars from the GAIA archive with alpha/Fe and Fe/H values")
print("Total:", len(gaiaExoplanetStarsWithMetallicities))
gaiaExoplanetStarsWithMetallicities.pprint(max_width=50000, max_lines=5000)


## Extract metallicities, calcluate galactic velocities from GAIA data

In [ ]:
##################################################################################
############### Analysis 1 - Metallicities and Galactic Velocities ###############
##################################################################################

# Create a copy of the star data for analysis 1
gaiaStarsAnalysis1 = gaiaExoplanetStarsWithMetallicities.copy()

# Filter on radial velocities
gaiaStarsAnalysis1 = gaiaStarsAnalysis1[(gaiaStarsAnalysis1['radial_velocity'].mask == False)]

# Calculate Fe/H errors
gaiaStarsAnalysis1['FeHErrorsUp'] = gaiaStarsAnalysis1['mh_gspphot_upper'] - gaiaStarsAnalysis1['mh_gspphot']
gaiaStarsAnalysis1['FeHErrorsLow'] = gaiaStarsAnalysis1['mh_gspphot'] - gaiaStarsAnalysis1['mh_gspphot_lower']

# Filter on Fe/H errors - This is done immediately to reduce calculating data which is ultimately never needed
gaiaStarsAnalysis1 = gaiaStarsAnalysis1[gaiaStarsAnalysis1['FeHErrorsUp'] < 0.04]
gaiaStarsAnalysis1 = gaiaStarsAnalysis1[gaiaStarsAnalysis1['FeHErrorsLow'] < 0.04]

# Calculate the Alpha/Fe errors
gaiaStarsAnalysis1['AlphaFEErrorsUp'] = gaiaStarsAnalysis1['alphafe_gspspec_upper'] - gaiaStarsAnalysis1['alphafe_gspspec']
gaiaStarsAnalysis1['AlphaFEErrorsLow'] = gaiaStarsAnalysis1['alphafe_gspspec'] - gaiaStarsAnalysis1['alphafe_gspspec_lower']

# Filter on Alpha/Fe errors
gaiaStarsAnalysis1 = gaiaStarsAnalysis1[gaiaStarsAnalysis1['AlphaFEErrorsUp'] < 0.2]
gaiaStarsAnalysis1 = gaiaStarsAnalysis1[gaiaStarsAnalysis1['AlphaFEErrorsLow'] < 0.2]

# Create the new columns
gaiaStarsAnalysis1['U'] = np.empty((len(gaiaStarsAnalysis1)))
gaiaStarsAnalysis1['UErr'] = np.empty((len(gaiaStarsAnalysis1)))
gaiaStarsAnalysis1['V'] = np.empty((len(gaiaStarsAnalysis1)))
gaiaStarsAnalysis1['VErr'] = np.empty((len(gaiaStarsAnalysis1)))
gaiaStarsAnalysis1['W'] = np.empty((len(gaiaStarsAnalysis1)))
gaiaStarsAnalysis1['WErr'] = np.empty((len(gaiaStarsAnalysis1)))
gaiaStarsAnalysis1['Speed'] = np.empty((len(gaiaStarsAnalysis1)))
gaiaStarsAnalysis1['SpeedErr'] = np.empty((len(gaiaStarsAnalysis1)))


# Go through all stars, calculate galactic velocities
for i in range (len(gaiaStarsAnalysis1)):
    gaiaStar = gaiaStarsAnalysis1[i]
    
    galacticVelocity = computeGalacticVelocity(gaiaStar)
    gaiaStarsAnalysis1['U'][i] = galacticVelocity[0]
    gaiaStarsAnalysis1['UErr'][i] = galacticVelocity[1]
    gaiaStarsAnalysis1['V'][i] = galacticVelocity[2]
    gaiaStarsAnalysis1['VErr'][i] = galacticVelocity[3]
    gaiaStarsAnalysis1['W'][i] = galacticVelocity[4]
    gaiaStarsAnalysis1['WErr'][i] = galacticVelocity[5]
    gaiaStarsAnalysis1['Speed'][i] = (galacticVelocity[0]**2 + galacticVelocity[2]**2 + galacticVelocity[4]**2)**(0.5)
    gaiaStarsAnalysis1['SpeedErr'][i] = (galacticVelocity[1]**2 + galacticVelocity[3]**2 + galacticVelocity[5]**2)**(0.5) # Is this even correct

# Set up metallicity axis
metallicityLabel = "[Fe/H] Metallicity"
metallicityData = gaiaStarsAnalysis1['mh_gspphot'].value
metallicityError = np.stack((gaiaStarsAnalysis1['FeHErrorsLow'].value, gaiaStarsAnalysis1['FeHErrorsUp'].value))

# Set up alpha/Fe axis
alphaOnFeLabel = "[Alpha/Fe] Metallicity"
alphaOnFeData = gaiaStarsAnalysis1['alphafe_gspspec'].value
alphaOnFeError = np.stack((gaiaStarsAnalysis1['AlphaFEErrorsLow'].value, gaiaStarsAnalysis1['AlphaFEErrorsUp'].value))

# Set up colour axis
colourLabel = "Galactic Speed [km/s]"
colourData = gaiaStarsAnalysis1['Speed'].value

plotScatterGraphOfData(szTitle=szGraphTitle, xAxis=metallicityLabel, yAxis=alphaOnFeLabel, xData=metallicityData, yData=alphaOnFeData, xError=metallicityError, yError=alphaOnFeError, logariseXAxis=False, logariseYAxis=False, colours=colourData, colourAxisLabel=colourLabel, sizes=np.array([]))
plt.show()

print("The pearson's coefficient of the data is %.3f with a %.3f 2-sided p-value " % (pearsonr(metallicityData, alphaOnFeData)[0], pearsonr(metallicityData, alphaOnFeData)[1]))
print("The spearman's rank coefficient of the data is %.3f with a %.3f 2-sided p-value " % (spearmanr(metallicityData, alphaOnFeData)[0], spearmanr(metallicityData, alphaOnFeData)[1]))
